1. [Load and Check Data](#1)
2. [Variable Description](#2)
    * [Drop Unused Labels](#3)
    * [Univeriate Variable Analysis](#4)
        - [Categorical Variable](#5)
3. [Basic Data Analysis](#6)
4. [Outlier Detection](#7)
5. [Missing Values](#8)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.figure_factory as ff
from tqdm import tqdm
import plotly.offline as py
import plotly.express as px
import seaborn as sns
import random
from collections import Counter
import warnings

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

<a id="1"></a>
# Load and Check Data

In [ ]:
cars_df = pd.read_csv("/kaggle/input/personal-cars-classifieds/all_anonymized_2015_11_2017_03.csv")

In [ ]:
cars_df.columns

In [ ]:
cars_df.head(10)

In [ ]:
cars_df.describe()

<a id="2"></a>
# Variable Description

<img src="https://i.ibb.co/zJp4dML/Ekran-Al-nt-s.png">
<img src="https://i.ibb.co/7rBvQ3C/Ekran-Al-nt-s.png">
<img src="https://i.ibb.co/FWtmhC2/Ekran-Al-nt-s.png">




1. maker : Company  name of producing the car like (Skoda)
1. model : Model of Car like (Octavia)
1. mileage: mil in KM
1. manufacture_year : manufacturing year of the car
1. engine_displacement : capacity of engine in ccm
1. engine_power : it is the maximum power that an engine can deliver in kW
1. body_type : type of car (this is discard)
1. color_slug : color of car (this is discard)
1. stk_year : year of the last emission control (48 + 37 = %85 is null) (this is discard)
1. transmission : Automatic or manual
1. door_count : # of door in the car
1. seat_count : # of seat in the car
1. fuel_type : Gasoline, diesel, cng , lpg and electric (Our target class) ( we eliminate Null values) (electric %1 and cng and lpg %0 this is problem, we eliminate them and we compare cars between (gasoline and diesel))
1. date_created : When the ad was scraped (this is discard)
1. date_last_seen :  when the ad was last seen. Our policy was to remove all ads older than 60 days (this is discard)
1. price_eur : list price converted to EUR

<a id="3"></a>
## DROP UNUSED LABELS

In [ ]:
cars_df.drop(labels = ["body_type","color_slug","stk_year","date_created","date_last_seen"],axis=1,inplace=True)


In [ ]:
print(cars_df[cars_df["fuel_type"].isnull()])
print("*"*40)
print(cars_df[cars_df["fuel_type"]== "electric"])
print("*"*40)
print(cars_df[cars_df["fuel_type"]== "lpg"])
print("*"*40)
print(cars_df[cars_df["fuel_type"]== "cng"])
print("*"*40)

Also we need to change fuel_type

In [ ]:
cars_df.drop(cars_df[cars_df["fuel_type"].isnull()].index,axis=0,inplace=True)
cars_df.drop(cars_df[cars_df["fuel_type"]== "electric"].index,axis=0,inplace=True)
cars_df.drop(cars_df[cars_df["fuel_type"]== "lpg"].index,axis=0,inplace=True)
cars_df.drop(cars_df[cars_df["fuel_type"]== "cng"].index,axis=0,inplace=True)

In [ ]:
fuel=cars_df["fuel_type"]
cars_df["fuel"] = [1 if i=="gasoline" else 0 for i in fuel]
# diesel 0
# gasoline 1

now continues with  check dataframe

In [ ]:
cars_df.columns[cars_df.isnull().any()]

In [ ]:
cars_df.isnull().sum()

So that only price is non null others are has none variable we find and eliminate or fill them

In [ ]:
cars_df.info()

* float64(5): price_eur,mileage,manufacture_year,engine_displacement,engine_power
* object(6): maker,model,transmission,door_count,seat_count,fuel_type

<a id="4"></a>
## Univeriate Variable Analysis

* Categorical Variable: Maker,Model,Transmission,Door_count,seat_count,fuel_type
* Numerical Variable: engine_displacement,engine_power,price_eur,mileage,manufacture_year

In [ ]:
def pie_plot(variable):
    """
        input: variable ex: "Model"
        output: pie plot & value count
    """
    #get feature
    var = cars_df[variable]
    #count number of categorical variable(value/sample)
    varValue = var.value_counts()
    #visualize
    fig = px.pie(values=varValue,names=varValue.index,template="plotly_white",title="Cars Pie Plot in {} column".format(variable))
    fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
    fig.show()
    print("{}:\n{}".format(variable,varValue))

<a id="5"></a>
### Categorical Variable

In [ ]:
categoricals = ["maker","model","transmission","door_count","seat_count","fuel_type"]
for c in categoricals:
    pie_plot(c)

<a id="6"></a>
# Basic Data Analysis
* Maker -- fuel
* Model -- fuel
* Transmission -- fuel
* Door_count -- fuel
* seat_count -- fuel
* fuel_type -- fuel

In [ ]:
# Maker vs fuel
cars_df[["maker","fuel"]].groupby(["maker"], as_index= False).mean().sort_values(by="fuel",ascending= False)

* %50 marker produces same fuel type automobiles like isuzu produce diesel or bently produce gasoline but some of marker has produce two of them like renult %51 produce gasoline %49 produce diesel automobiles

In [ ]:
# model vs fuel
pd.set_option('display.max_rows',1000) #we see all features
cars_df[["model","fuel"]].groupby(["model"], as_index= False).mean().sort_values(by="fuel",ascending= False)

Same with marker but model is more spesific feture also we extract new feature with using model and marker 

In [ ]:
# Transmision vs fuel
cars_df[["transmission","fuel"]].groupby(["transmission"], as_index= False).mean().sort_values(by="fuel",ascending= False)

* manual automobiles much more gasoline and automatics are more diesel but it is too close each other we directly this feature in ML

In [ ]:
# door_count vs fuel
cars_df[["door_count","fuel"]].groupby(["door_count"], as_index= False).mean().sort_values(by="fuel",ascending= False)

* it is also good feature for extract fuel type but we change door_count in feature engineering

In [ ]:
# seat_count vs fuel
cars_df[["seat_count","fuel"]].groupby(["seat_count"], as_index= False).mean().sort_values(by="fuel",ascending= False)

* this is another important feature for classification but also we need to change in feature engineering

In [ ]:
# fuel_type vs fuel
cars_df[["fuel_type","fuel"]].groupby(["fuel_type"], as_index= False).mean().sort_values(by="fuel",ascending= False)

also this is a joke :) because they are same thing we control our methodologies is good ways

<a id="7"></a>
# Outlier Detection

In [ ]:
def detect_outliers(df,features):
    outlier_indices = []
    for c in features:
        #1st Quartile
        Q1 = np.percentile(df[c],25)
        #3rd Quartile
        Q3 = np.percentile(df[c],75)
        #IQR
        IQR = Q3-Q1
        #Outlier Step
        outlier_step = IQR*1.5
        #Detect Outlier and their Indeces
        outlier_list_col = df[(df[c]< Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # Store indeces
        outlier_indices.extend(outlier_list_col),
        
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i,v in outlier_indices.items() if v > 2)
    return multiple_outliers

In [ ]:
cars_df.loc[detect_outliers(cars_df,["engine_displacement","engine_power","price_eur","mileage","manufacture_year"])]

In [ ]:
cars_df = cars_df.drop(detect_outliers(cars_df,["engine_displacement","engine_power","price_eur","mileage","manufacture_year"]), axis=0).reset_index(drop = True)

<a id="8"></a>
# Missing Values

In [ ]:
cars_df.columns[cars_df.isnull().any()]

In [ ]:
cars_df.isnull().sum()

try to fill missing values

In [ ]:
#pd.set_option('display.max_rows',421755) if you want to  check all of them is NaN with your eyes open this code and delete last .isnull().all()
cars_df["model"][cars_df["maker"].isnull()].isnull().all()

In [ ]:
cars_df[cars_df["maker"].isnull()]

so I understand that there is no information about NaN values of model or check other feature I check with my eyes I delete all NaN models

In [ ]:
cars_df.drop(cars_df[cars_df["maker"].isnull()].index,axis=0,inplace=True)

In [ ]:
cars_df.isnull().sum()

In [ ]:
cars_df["maker"][cars_df["model"].isnull()].value_counts()

Firstly 46 different Maker Models are fill most of them is mercedes bmw and audi and same known makers we fill the basic ones like infinity I check infinity cars and also fill the car

In [ ]:
cars_df["model"][(cars_df.maker=="infinity") & (cars_df.fuel==1)].value_counts()

also I add this fill area wit fx35 model

In [ ]:
cars_df["model"][(cars_df.maker=="infinity") & (cars_df.model.isnull())].index

In [ ]:
cars_df["model"][3393719] = "fx35"

this is only one model fill 

for aston-martin

In [ ]:
cars_df[(cars_df.maker=="aston-martin")]

also we check that 2012> rapide 2009 < v8-vantage produce we fill them NaN values

In [ ]:
cars_df["model"][3350774] = "rapide"
cars_df["model"][3350765] = "rapide"
cars_df["model"][3350756] = "rapide"
cars_df["model"][3350747] = "v8-vantage"

In [ ]:
cars_df[(cars_df.maker=="aston-martin")]

uff we fill 4 values now continue with rolls-roles now you now that is terrible hard work now we write automatically fill the model values because we fill it with other features. but firstly we check tghat

but we dont use visualization techniques because we are too many categorical variables and cpu is not enough for vizualization I predict that year maker and fuel type is categori for model

In [ ]:
index_nan_model = list(cars_df["model"][cars_df["model"].isnull()].index)

In [ ]:
model_med = cars_df["model"].value_counts().idxmax()
model_med

In [ ]:
def fill_nulls(fill_column,other_column1,other_column2,other_column3,idx):
    pred = cars_df[fill_column][(( cars_df[other_column1] == cars_df.iloc[idx][other_column1]) &
                             ( cars_df[other_column2] == cars_df.iloc[idx][other_column2]) &
                             ( cars_df[other_column3] == cars_df.iloc[idx][other_column3]))].value_counts()

    if not pred.empty:
        print(idx,"degisti")
        cars_df["model"].iloc[idx]= pred.idxmax()
    else:
        print(idx,"silindi")
        cars_df.drop(idx,axis=0,inplace=True)

In [ ]:
"""
for i in tqdm(index_nan_model:
    fill_nulls("model","maker","fuel","manufacture_year",i)
"""

<img src="https://i.ibb.co/f9mnTFx/Ekran-Al-nt-s.png"> after 20 hours we can continue with new data set

In [ ]:
cars_df.to_csv("temizleme2.csv",index=False)

In [ ]:
cars_df = pd.read_csv("/kaggle/input/temizleme/temizleme2.csv",low_memory=False)

In [ ]:
cars_df.isnull().sum()

We will drop over 100.000 nulll data because we don't have enough time for cleaning the data

In [ ]:
cars_df.drop(cars_df[cars_df["engine_displacement"].isnull()].index,axis=0,inplace=True)
cars_df.drop(cars_df[cars_df["engine_power"].isnull()].index,axis=0,inplace=True)
cars_df.drop(cars_df[cars_df["transmission"].isnull()].index,axis=0,inplace=True)
cars_df.drop(cars_df[cars_df["door_count"].isnull()].index,axis=0,inplace=True)
cars_df.drop(cars_df[cars_df["seat_count"].isnull()].index,axis=0,inplace=True)

In [ ]:
cars_df.isnull().sum()

In [ ]:
cars_df.to_csv("temizleme3.csv",index=False)

we have 650k data after the cleaning job

Fill Mileage

In [ ]:
cars_df[cars_df["mileage"].isnull()]

In [ ]:
cars_df["model"][cars_df["mileage"].isnull()].value_counts()

In [ ]:
cars_df["mileage"][(cars_df.model=="g400-cdi") & (cars_df.mileage.isnull())].index

In [ ]:
cars_df[(cars_df.model=="g400-cdi") & (cars_df.manufacture_year <= 1990)]

In [ ]:
"""
from tqdm import tqdm
index_nan_mileage = list(cars_df["mileage"][cars_df["mileage"].isnull()].index)
mileage_med = cars_df["mileage"].median()
for i in tqdm(index_nan_mileage):
    mileage_pred = cars_df["mileage"][((cars_df["manufacture_year"] == cars_df.iloc[i]["manufacture_year"]) &(cars_df["model"] == cars_df.iloc[i]["model"]))].median()
    if not np.isnan(mileage_pred):
        print("Change value")
        cars_df["mileage"].iloc[i] = mileage_pred   
    else:
        print("Average value")
        cars_df["mileage"].iloc[i] = mileage_med
"""

In [ ]:
cars_df.to_csv("temizleme4.csv",index=False)

In [ ]:
"""
from tqdm import tqdm
index_nan_manufacture_year = list(cars_df["manufacture_year"][cars_df["manufacture_year"].isnull()].index)
manufacture_year_med = cars_df["manufacture_year"].median()
for i in tqdm(index_nan_manufacture_year):
    manufacture_year_pred = cars_df["manufacture_year"][((cars_df["model"] == cars_df.iloc[i]["model"])
                                                         & (cars_df["mileage"] == cars_df.iloc[i]["mileage"])
                                                        )].median()
    if not np.isnan(manufacture_year_pred):
        print("Change value")
        cars_df["mileage"].iloc[i] = manufacture_year_pred   
    else:
        print("Average value")
        cars_df["mileage"].iloc[i] = manufacture_year_med
"""

In [ ]:
cars_df = pd.read_csv("/kaggle/input/temizleme/temizleme4.csv",low_memory=False)

In [ ]:
cars_df.isnull().sum()

Now our data set has not null variable so that we can use it for solve problem